In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,Covid live updates: States expand Covid vaccin...,https://www.nbcnews.com/news/us-news/live-blog...,"Tue, 12 Jan 2021 16:52:30 GMT",2021-01-12 16:52:30+00:00
5,California's Disneyland to become Covid vaccin...,https://www.bbc.com/news/world-us-canada-55636323,"Tue, 12 Jan 2021 15:54:00 GMT",2021-01-12 15:54:00+00:00
2,UK science advisers: publish evidence behind C...,https://www.nature.com/articles/d41586-021-000...,"Tue, 12 Jan 2021 15:50:15 GMT",2021-01-12 15:50:15+00:00
1,Immunological characteristics govern the trans...,https://science.sciencemag.org/content/early/2...,"Tue, 12 Jan 2021 15:30:42 GMT",2021-01-12 15:30:42+00:00
4,Billions will not receive a COVID-19 vaccine i...,https://reliefweb.int/report/world/billions-wi...,"Tue, 12 Jan 2021 12:31:00 GMT",2021-01-12 12:31:00+00:00
7,Hillsborough County launches new COVID-19 vacc...,https://www.wfla.com/community/health/coronavi...,"Tue, 12 Jan 2021 11:32:00 GMT",2021-01-12 11:32:00+00:00
3,How COVID unlocked the power of RNA vaccines ...,https://www.nature.com/articles/d41586-021-000...,"Tue, 12 Jan 2021 11:02:05 GMT",2021-01-12 11:02:05+00:00
8,COVID-19 Vaccine NYU Langone Health,https://nyulangone.org/locations/covid-19-vaccine,"Tue, 12 Jan 2021 00:51:25 GMT",2021-01-12 00:51:25+00:00
9,UN: COVID-19 herd immunity unlikely in 2021 de...,https://www.modernhealthcare.com/safety/un-cov...,"Mon, 11 Jan 2021 21:08:23 GMT",2021-01-11 21:08:23+00:00
6,South Carolinians Aged 70 and Older Can Schedu...,https://scdhec.gov/news-releases/south-carolin...,"Mon, 11 Jan 2021 17:26:15 GMT",2021-01-11 17:26:15+00:00
